In [9]:
import pandas as pd
import numpy as np
import mlflow.sklearn
import mlflow
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import f1_score,accuracy_score,precision_score,recall_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import string

In [2]:

import dagshub
dagshub.init(repo_owner='yadavdipu296', repo_name='mlops', mlflow=True)
mlflow.set_tracking_uri("https://dagshub.com/yadavdipu296/mlops.mlflow")



Accessing as yadavdipu296

Initialized MLflow to track repo "yadavdipu296/mlops"

Repository yadavdipu296/mlops initialized!

In [ ]:

# Load the data
# Load the data
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])


# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

# Normalize the text data
df = normalize_text(df)

x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


<>:35: SyntaxWarning: invalid escape sequence '\s'
<>:35: SyntaxWarning: invalid escape sequence '\s'
/var/folders/g_/sy0vrt390g59cw_vcq84l2br0000gn/T/ipykernel_2169/2280027822.py:35: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()
/var/folders/g_/sy0vrt390g59cw_vcq84l2br0000gn/T/ipykernel_2169/2280027822.py:63: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})
2025/08/04 17:05:53 INFO mlflow.tracking.fluent: Experiment with name 'LoR Hyperparameter Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/3a67ec9480e044118615a2121b22988c', creation_time=1754307354210, experiment_id='3', last_update_time=1754307354210, lifecycle_stage='active', name='LoR Hyperparameter Tuning', tags={}>

In [8]:

# Set the experiment name
mlflow.set_experiment("LoR Hyperparameter Tuning")

<Experiment: artifact_location='mlflow-artifacts:/3a67ec9480e044118615a2121b22988c', creation_time=1754307354210, experiment_id='3', last_update_time=1754307354210, lifecycle_stage='active', name='LoR Hyperparameter Tuning', tags={}>

In [10]:
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']

}

model=LogisticRegression()

In [14]:
with mlflow.start_run(run_name='logistic regression with bow'):
    grid_search=GridSearchCV(model,param_grid,cv=5,scoring='accuracy',n_jobs=-1)
    grid_search.fit(X_train,y_train)

    for params,scores,std_scores in zip(grid_search.cv_results_['params'], grid_search.cv_results_['mean_test_score'], grid_search.cv_results_['std_test_score']):
        with mlflow.start_run(run_name=f"LR with params: {params}", nested=True):
            model = LogisticRegression(**params)
            model.fit(X_train, y_train)
            
            # Model evaluation
            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            
            # Log parameters and metrics
            mlflow.log_params(params)
            mlflow.log_metric("mean_cv_score", scores)
            mlflow.log_metric("std_cv_score", std_scores)
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1_score", f1)

            print(f"Mean CV Score: {scores}, Std CV Score: {std_scores}")
            print(f"Accuracy: {accuracy}")
            print(f"Precision: {precision}")
            print(f"Recall: {recall}")
            print(f"F1 Score: {f1}")


          # Log the best run details in the parent run
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    mlflow.log_params(best_params)
    mlflow.log_metric("best_f1_score", best_score)
    
    print(f"Best Params: {best_params}")
    print(f"Best F1 Score: {best_score}")

    # Save and log the notebook

    # Log model
    mlflow.sklearn.log_model(grid_search.best_estimator_, "model")
            


Mean CV Score: 0.7299690625068085, Std CV Score: 0.013317073170155872
Accuracy: 0.7397590361445783
Precision: 0.7752027809965237
Recall: 0.6591133004926109
F1 Score: 0.7124600638977636
Mean CV Score: 0.7804568000755282, Std CV Score: 0.009999128328532197
Accuracy: 0.7893975903614457
Precision: 0.7805825242718447
Recall: 0.7921182266009852
F1 Score: 0.7863080684596577
Mean CV Score: 0.7803361729013705, Std CV Score: 0.0063196819891571405
Accuracy: 0.7826506024096386
Precision: 0.77431906614786
Recall: 0.7842364532019704
F1 Score: 0.7792462065589819
Mean CV Score: 0.7892532153932185, Std CV Score: 0.010046277276775439
Accuracy: 0.7951807228915663
Precision: 0.7864077669902912
Recall: 0.7980295566502463
F1 Score: 0.7921760391198044
Mean CV Score: 0.7711789654095587, Std CV Score: 0.008539026441741348
Accuracy: 0.7860240963855422
Precision: 0.7835153922542205
Recall: 0.7773399014778325
F1 Score: 0.7804154302670623
Mean CV Score: 0.7782868907819342, Std CV Score: 0.004522890891736139
Accura